In [1]:
import pandas as pd
import json, requests, re
from bs4 import BeautifulSoup as bs
from urllib.parse import unquote
from os.path import join, abspath
from codecs import open

In [2]:
df = pd.read_csv(abspath(join('..', 'dataset', 'map_label_to_dataset.csv')))

In [3]:
# drop duplicate of class 1
useful_df = df.where(df["class"]==1).dropna()
useful_df = useful_df.replace(to_replace=r'pic\.twitter\.com/[A-Za-z0-9]{10}', value='', regex=True)
useful_df = useful_df.drop_duplicates(subset='text')
useful_df

,tweet_id,text,class
0,"""1259128006158585857""",ไขข้องใจเกษตรกรไร้เอกสารสิทธิ์ สามารถยื่นขอรับ...,1.0
1,"""1259127592671604736""",เงินเริ่มร่อยหรอ ประกันสังคม เคาะแล้ว! ไม่จ่าย...,1.0
2,"""1259127373594673152""",ไขข้องใจเกษตรกรไร้เอกสารสิทธิ์ สามารถยื่นขอรับ...,1.0
3,"""1259126509975097346""",บุคลากรในทำเนียบขาวติดเชื้อ #โควิด19 กันเพียบ ...,1.0
6,"""1259117165711626241""",“ปธน.ทรัมป์” ไม่หวั่น โควิด-19 ระบาดในทำเนียบข...,1.0
...,...,...,...
995,"""1255367589989158914""",ตัวเลขสหรัฐตายจาก โควิด-19 มากกว่าที่ตายในสงคร...,1.0
996,"""1255366614096191489""","""หมอทวีศิลป์"" ยืนยัน คุณแม่ลูกอ่อน สามารถให้นม...",1.0
997,"""1255366199522648065""",รัฐบาล #เยอรมนี ตัดสินใจให้ความช่วยเหลือสายการ...,1.0
998,"""1255366095550263298""",เปิดร้านค้าขายอีกครั้งได้เมื่อไหร่ ในสถานการณ์...,1.0


In [4]:
# drop duplicate of class 0
useless_df = df.where(df["class"]==0).dropna()
useless_df = useless_df.replace(to_replace=r'pic\.twitter\.com/[A-Za-z0-9]{10}', value='', regex=True)
useless_df = useless_df.drop_duplicates(subset='text')
useless_df

,tweet_id,text,class
4,"""1259119806726287360""",คัดให้ลูกค้าอย่างดีทุกลูก ขอบคุณลูกค้าที่อุดหน...,0.0
5,"""1259118774361272320""","ผบ.ตร.สั่งการเด็ดขาดคดี""5ครู2ศิษย์""แต่ได้ประกั...",0.0
11,"""1259093195759087616""",ชุดเซทเซทละ180 ค่าส่ง40บาทค่ะ มีอีกหลายลายหลา...,0.0
12,"""1259084995244617728""",แมสพร้อมส่ง ทักมาสอบถามก่อนได้นะคะ\n#carrymina...,0.0
14,"""1259082005645365253""",ทำไมเปิดร้านแล้ว ทำฉากกั้น จัดโต๊ะห่างกัน ถึงย...,0.0
...,...,...,...
970,"""1255404836243279879""",River View Protect & Cleaning เข้าปฏิบัติการฉี...,0.0
971,"""1255403529872777218""",รับเช่าพระเครื่องใหม่-เก่าส่งรูปประเมินราคา 24...,0.0
977,"""1255394750103990272""",Ethyl Alcohol Spray 75%\nแอลกอฮอล์ล้างมือ และท...,0.0
984,"""1255380715996413952""",เจ้าของร้านขายส่งชุดแต่งงาน หันเย็บหน้ากากผ้า ...,0.0


In [5]:
# use this dataset for prevent imbalance dataset problem
emotional_df = pd.read_csv(abspath(join('..', 'dataset', 'map_emotional_dataset.csv')))
emotional_df = emotional_df.replace(to_replace=r'pic\.twitter\.com/[A-Za-z0-9]{10}', value='', regex=True)
emotional_df = emotional_df.drop_duplicates(subset='text')
emotional_df

,tweet_id,text,class
0,"""1260234474668744709""",facebookขึ้นอันนี้มา.... ไปแนะนำให้พ่อมึงนะมา...,0
1,"""1259850883875078144""",สมองคุณคิดได้แค่นี้ก็ดีแล้วที่ลาออกมานะ\nอย่าเ...,0
2,"""1259796010957590528""",รีวิวคอร์สหน่อยสิ คนที่เข้าอบรมคือใคร มีฐานะเด...,0
3,"""1259709426052681728""",ฝาก 500 รับทันที 500 ไม่ต้องทำเทิร์น\n#บาคาร่า...,0
4,"""1259500710833475586""",มารีวิวเพื่อนที่เรียนกับอี #ผู้กองเบนซ์ กูบ...,0
...,...,...,...
595,"""1258588812655001602""",หลอกคนอย่างเป็นระบบ​ เราเห็นมาพักนึงแล้ว​ ไม่ค...,0
596,"""1258588550183809025""",ขออนุญาต​ฝากร้านนะคะ\nขนมไข่เตาถ่าน สูตรแป้งน้...,0
597,"""1258588395640549378""",แต้มบุญไม่ถึงไม่ได้เป็นหรอกคร่า อย่ามั่น #ผู้ก...,0
598,"""1258588056057241605""",หล่อดีนะแต่ตรรกะคอนเซอเวทีฟ ดูความคิดจมปรัก #ผ...,0


In [6]:
# utils function
def tweets_generator(emotional_dataset=False):
    file_name = 'emotional_raw_dataset.json' if emotional_dataset else 'covid19_raw_dataset.json'
    with open(
              abspath(join('..', 'dataset', file_name)),
              'r',
              encoding='utf-8-sig') as f:
        tweets = json.load(f)
    for index in range(len(tweets)):
        tweets[index]['text_html'] = unquote(tweets[index]['text_html'])
        yield tweets[index]
def get_tweet(tweet_id, tweets):
    for tweet in tweets:
        if tweet['tweet_id'] == tweet_id:
            return tweet
    else:
        raise ValueError(f'Invalid tweet_id: {tweet_id}')

In [7]:
useful_tweets, useless_tweets = [], []
covid_tweets, emotional_tweets = list(tweets_generator()), list(tweets_generator(True))
for tweet_id in useful_df.loc[:, 'tweet_id']:
    tweet = get_tweet(tweet_id.replace('"', ''), covid_tweets)
    useful_tweets.append(tweet)
for tweet_id in useless_df.loc[:, 'tweet_id']:
    tweet = get_tweet(tweet_id.replace('"', ''), covid_tweets)
    useless_tweets.append(tweet)
# add emotional dataset as class 0 for prevent imbalance dataset problem
for tweet_id in emotional_df.loc[:, 'tweet_id']:
    tweet = get_tweet(tweet_id.replace('"', ''), emotional_tweets)
    useless_tweets.append(tweet)
    if len(useless_tweets) == len(useful_tweets):
        break

In [8]:
print(f'useful = {len(useful_tweets)}, useless = {len(useless_tweets)}, ratio = {len(useful_tweets)/len(useless_tweets)}')

useful = 719, useless = 719, ratio = 1.0


In [9]:
def get_url_title(urls):
    titles = []
    for url in urls:
        try:
            res = requests.get(url)
            soup = bs(res.content, 'lxml')
            titles.append(soup.select_one('title').text)
        except: # sometimes the title is None will cause error
            pass
    return titles
def clean_text(tweet):
    noise_cut_pattern = r'[^a-zA-Z0-9ก-๙| ]+'
    cleaned_text = ''
    soup = bs(tweet['text_html'], 'html.parser')
    try:
        text_list = list(
                filter(lambda txt: not (str(txt).startswith('<a') or 
                                        str(txt).startswith('<img') or 
                                        str(txt).startswith('<span')),
                        soup.p.contents))
    except:
        raise ValueError(tweet['tweet_id'])
    if len(tweet['links']) > 0:
        # if the tweet has a link, then transform a link to webpage title to get more information
        titles = get_url_title(tweet['links'])
        text_list += titles
    # merge text inside text_list into a string
    for txt in text_list:
        txt = str(txt)
        if txt == '' or txt == ' ' or txt == None:
            continue
        cleaned_text += ' ' + txt.replace('<strong>', '').replace('</strong>', '')
    # clean special characters that has no meaning
    cleaned_text = re.sub(noise_cut_pattern, '', cleaned_text)
    return cleaned_text

In [10]:
# clean hashtags and links in tweet-text
for index in range(max(len(useful_tweets), len(useless_tweets))):
    if index < len(useful_tweets):
        useful_tweets[index]['text'] = clean_text(useful_tweets[index])
    if index < len(useless_tweets):
        useless_tweets[index]['text'] = clean_text(useless_tweets[index])

In [11]:
# save useful tweets as useful-tweets.json
with open(abspath(join('..', 'dataset', 'useful-tweets.json')), 'w', encoding='utf-8-sig') as f:
    json.dump(useful_tweets, f, ensure_ascii=False, indent=3)
# save useless tweets as useless-tweets.json
with open(abspath(join('..', 'dataset', 'useless-tweets.json')), 'w', encoding='utf-8-sig') as f:
    json.dump(useless_tweets, f, ensure_ascii=False, indent=3)